Zunächst müssen wir das OdeNet-Modul importieren

In [1]:
from odenet import *


Mit OdeNet.word_info() greift man auf Lexikon-Einträge zu, bekommt die Lexikon-ID für ein Wort, den Lemma-Wert, POS und die IDs der Synsets, in denen das Wort enthalten ist.

In [2]:
OdeNet.word_info("Auseinandersetzung")

Auseinandersetzung n 
SENSE: odenet-1163-n  ('i110406', [], 'Zustand der Opposition zwischen Personen, Ideen oder Interessen.', [('hyponym', 'odenet-26329-n'), ('hyponym', 'odenet-35625-n'), ('hyponym', 'odenet-8610-n'), ('hyponym', 'odenet-13574-n'), ('hyponym', 'odenet-25988-n')], ['Gerangel', 'Krach', 'Clinch', 'Streit', 'Zankerei', 'Zerwürfnis', 'Zoff', 'Differenzen', 'Zank', 'Stunk', 'Querelen', 'Streiterei', 'Knatsch', 'Rangelei', 'Streitigkeit', 'Hader', 'Hickhack', 'Zwist', 'Zwistigkeit', 'Geplänkel', 'Hakelei', 'Wickel', 'Klinsch', 'Auseinandersetzung', 'Scharmützel'])

SENSE: odenet-5915-n  ('i40429', [], 'Kleiner kurzfristiger Streit.', [('hypernym', 'odenet-3632-n')], ['Schererei', 'Differenzen', 'Querelen', 'Auseinandersetzung', 'Streitigkeiten', 'Zerwürfnisse', 'Reibereien', 'Dispute', 'Brüche'])

SENSE: odenet-9925-n  ('i74403', [], 'Dialog mit großer Nicht-Übereinstimmung.', [('hyponym', 'odenet-471-n'), ('hyponym', 'odenet-6948-n'), ('hyponym', 'odenet-11443-n'), ('hyp

Hier bekommt man die Lexikon-IDs für eine Liste von Wörtern.

In [3]:
wordlist = ['Frühling','Sommer','Herbst','Winter']
for w in wordlist:
    print(w + ': ' + OdeNet.word_id(w))

Frühling: w14145
Sommer: w44811
Herbst: w1202374
Winter: w25612


Mit check_word_id bekommt man für eine Lexikon-ID Lemma, POS, Synsets und Relationen

In [4]:
check_word_id('w14145')

LEMMA: Frühling
POS: n
SENSE ID: w14145_3067-n


Mit words_in_synset bekommt man die Wörter, die in einem Synset sind.

In [10]:
words_in_synset('odenet-9050-v')

['tarnen',
 'verbergen',
 'kaschieren',
 'verstecken',
 'maskieren',
 'verhehlen',
 'verheimlichen',
 'den Blicken entziehen',
 'camouflieren']

Mit check_synset bekommt man alle Informationen zu einem Synset.

In [6]:
check_synset("odenet-2982-n")

('i74188',
 [],
 'Dialog mit kontroversen Ansichten.',
 [('hyponym', 'odenet-26556-n'),
  ('hyponym', 'odenet-29356-n'),
  ('hyponym', 'odenet-2743-n'),
  ('hyponym', 'odenet-7533-n'),
  ('hypernym', 'odenet-2801-n')],
 ['Auseinandersetzung',
  'Diskussion',
  'Debatte',
  'Wortstreit',
  'Disputation',
  'Disput',
  'Streitgespräch'])

In [7]:
hypernyms_word("überlassen")

[('w9034_13770-v',
  'odenet-10390-n',
  ['Liegenschaft', 'Grundbesitz', 'Landbesitz']),
 ('w9034_15292-v', 'odenet-21765-n', ['Schulden', 'Fremdkapital'])]

In [8]:
meronyms_word("Morgenland")

[('w12715_2720-n',
  'odenet-14514-n',
  ['Saudi-Arabien', 'Königreich Saudi-Arabien', 'Saudisch-Arabien']),
 ('w12715_2720-n', 'odenet-14519-n', ['Syrien', 'Arabische Republik Syrien']),
 ('w12715_2720-n', 'odenet-14516-n', ['Türkei', 'Republik Türkei']),
 ('w12715_9897-n', 'odenet-9309-n', ['Ferner Osten', 'Fernost'])]

In [9]:
myword = "zeigen"
(lemma_id, lemma_value, pos, senses) = check_word_lemma(myword)
print (lemma_value + " " + pos + " ")
for sense in senses:
    print("SENSE: " + str(sense[1]) + "  " + str(check_synset(sense[1])) + "\n")
print("HYPERNYMS: " + str(hypernyms_word(myword)))
print("HYPONYMS: " + str(hyponyms_word(myword)))
print("MERONYMS: " + str(meronyms_word(myword)))
print("HOLONYMS: " + str(holonyms_word(myword)))
print("ANTONYMS: " + str(antonyms_word(myword)))


zeigen v 
SENSE: odenet-533-v  ('i33216', [], 'Vorschlagen, etwas für jemanden zu tun.', [('hypernym', 'odenet-8644-v')], ['andienen', 'anbieten', 'bieten', 'zeigen'])

SENSE: odenet-3065-v  ('i25556', 'bring forward and present to the mind', [], [('hypernym', 'odenet-9246-v')], ['zeigen', 'präsentieren', 'darstellen', 'vorzeigen', 'abbilden', 'vorführen', 'demonstrieren', 'vorstellen'])

SENSE: odenet-4206-v  ('i32440', 'make visible or noticeable', [], [('hyponym', 'odenet-2339-v'), ('hyponym', 'odenet-11159-v'), ('hyponym', 'odenet-1192-v'), ('hyponym', 'odenet-22914-v'), ('hyponym', 'odenet-2770-v')], ['kundtun', 'zeigen', 'offenbaren', 'bekunden'])

SENSE: odenet-5328-v  ('i25048', 'confirm the truth of', [], [('hypernym', 'odenet-12792-v')], ['bestätigen', 'zeigen', 'aufzeigen', 'nachweisen', 'belegen', 'Nachweis erbringen', 'Beweis erbringen für', 'beweisen'])

SENSE: odenet-9416-v  ('i26697', 'make reference to', [], [('hyponym', 'odenet-36186-v')], ['zeigen', 'erwähnen', 'ange